# Homework 1 (Due Thursday, November 3rd, 2022 at 6:29pm PST)

Every day late is -10%.

You are a business analyst working for a major US toy retailer. A manager in the marketing department would like your help to build a classification model that will predict whether a review is positive or negative. use the `../datasets/good_amazon_toy_reviews.txt` and `../datasets/poor_amazon_toy_reviews.txt` datasets for this exercise.

Combine the good and the poor datasets together.

### Preprocessing and Regex (3pts)

Perform the following cleanup steps:

* There are malformed characters in the review text. For instance, notice the `&#34;` - these are examples of incorrectly decoded [HTML encodings](https://krypted.com/utilities/html-encoding-reference/).
```
"amazing quality first of all, these cards are amazing proxies (but don't try to use em in &#34;official duels&#34; unless a judge is okay with it, if you have the real thing to show) and look amazing in your binder!"
```
Please clean up all instances of these incorrect decodings.

* Use **regular expressions to parse out and normalize all references to recipients (children, spouses, parents, etc.) and gift occasions (Christmas, birthdays, and anniversaries)**, and account for the possibility that people may spell words "son" / "children" / "Christmas" as both singular and plural, upper or lower-cased.

### Vectorization (7pts)

* with/without stopword removal
* with 1) no stemming or lemmatization, 2) stemming, 3) lemmatization
* using `TfIdfVectorizer` versus `CountVectorizer`
* using `ngram` sizes of 1, 2, and 3

Edit: Perform vectorization using above instructions. Print out the shape of the final vectorized datasets.

**Submit everything as a new notebook and Slack direct message to me (Yu Chen) and the TAs the HW as an attachment.**

**NOTE**: Name the notebook `lastname_firstname_HW1.ipynb`.

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## Part1: Preprocessing and Regex

In [2]:
# create dataframe & Concate the good and the poor datasets
df1 = pd.DataFrame( open("../datasets/good_amazon_toy_reviews.txt", "r"), columns=["review"])
df2 = pd.DataFrame( open("../datasets/poor_amazon_toy_reviews.txt", "r"), columns=["review"])
df = pd.concat([df1, df2]).reset_index(drop = True)
print(df.shape)
df.head()

(114917, 1)


,review
0,Excellent!!!\n
1,"""Great quality wooden track (better than some ..."
2,my daughter loved it and i liked the price and...
3,Great item. Pictures pop thru and add detail a...
4,I was pleased with the product.\n


In [3]:
# create function to clean up the review
def Cleanup(text):
    # remove malformed characters in the review text
    #cleaned_text = html.unescape(text).strip().lower()
    cleaned_text = text.strip().lower()
    cleaned_text = re.sub(r'(&#[0-9]+)|(%(\w)+)', '', cleaned_text)
    cleaned_text = re.sub(r'(\<(br) \/\>)(\<(br) \/\>)?', ' ', cleaned_text)
    cleaned_text = re.sub('['+string.punctuation+']', '', cleaned_text)
    
    # Use regular expressions to parse out and normalize all references to recipients and gift occasions 
    cleaned_text = re.sub(r"\b((b(irth)?day)( party| parties)?|anniversary|anniversaries|christmas|halloween( party| parties)?|thanksgiving|valentine's day|father's day|mother's day|housewarming( party| parties)?)\b", 
                        "_OCCASION_", cleaned_text)
    cleaned_text = re.sub(r"\b(my |our )?([a-zA_Z0-9]+ ?(y(ea)?rs?(-| ?)(olds?)?|y\.?o\.?|months?(-| ?)olds?) )?(younger )?(older )?(twin )?(daughters?|sons?|child|children|kids?|husbands?|wifes?|(little )?girls?|(little )?boys?|mom|mother|dad|father|grand ?sons?|grand ?daughters?|grand ?child|grand ?children|grand ?kids?|girlfriend|boyfriend|honey|baby|babies|sisters?|brothers?|aunts?|uncles?|cousins?|fiances?|parents?|friends?|classmates?|co\-?workers?|nieces?|nephews?)\b", 
                        "_RECIPIENT_", cleaned_text)
    cleaned_text = re.sub(r"\b(my |our )([a-zA_Z0-9]+ (years?-olds?|years? ?olds?|yr\.? olds?|yrs?|y\.?o\.?|months?-olds?|months? olds?))( and )?([a-zA_Z0-9]+ (years?-olds?|years? ?olds?|yr\.? olds?|yrs?|y\.?o\.?|months?-olds?|months? olds?))?\b", 
                        "_RECIPIENT_", cleaned_text)
    
    return cleaned_text

In [4]:
df["review"] = df.apply(lambda x: Cleanup(x["review"]), axis=1)

In [5]:
df.head()

,review
0,excellent
1,great quality wooden track better than some ot...
2,_RECIPIENT_ loved it and i liked the price and...
3,great item pictures pop thru and add detail as...
4,i was pleased with the product


## Part 2: Perform vectorization 

In [6]:
# prep nltk
nltk_stopwords = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [7]:
def Preprocess(Stopword_Removal, Stemming, Lemmatization, text):
    word_tokens = nltk.word_tokenize(text)
    
    # stopword removal
    if Stopword_Removal == True:
        word_remove_stopwords = []
        for t in word_tokens:
            if t in nltk_stopwords:
                continue
            word_remove_stopwords.append(t)
        word_tokens = word_remove_stopwords
    
    # stemming 
    if Stemming == True:
        word_tokens = [stemmer.stem(t) for t in word_tokens]
    
    # lemmatization
    if Lemmatization == True:
        word_tokens = [lemmatizer.lemmatize(t) for t in word_tokens]
        
    cleaned_review = " ".join(word_tokens)
        
    return cleaned_review

In [8]:
# Vectorization
def Vectorize(Vectorization, Ngram, text):
    if Vectorization == 'COUNT':
        vectorizer = CountVectorizer(ngram_range=Ngram)
        X = vectorizer.fit_transform(text)
        Count = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
        
        return Count
        
    elif Vectorization == 'TFIDF':
        vectorizer = TfidfVectorizer(ngram_range=Ngram)
        corpus = list(text.values)
        X = vectorizer.fit_transform(corpus)
        terms = vectorizer.get_feature_names()
        tf_idf = pd.DataFrame(X.toarray(), columns=terms)
        
        return tf_idf

In [9]:
# Parameters
instructions = [[False, False, False], [False, True, False], [False, False, True], [True, False, False],
                [True, True, False], [True, False, True]]
vectorize = [['COUNT', (1,1)], ['COUNT', (2,2)], ['COUNT', (3,3)], 
             ['TFIDF', (1,1)], ['TFIDF', (2,2)], ['TFIDF', (3,3)]]

In [10]:
# Sample for only 20,000 rows 
df2 = df.loc[:19999, ['review']]
df2.shape

(20000, 1)

In [11]:
for Stopword_Removal, Stemming, Lemmatization in instructions:
    for Vectorization, Ngram in vectorize:
        df2["review"] = df2.apply(lambda x: Preprocess(Stopword_Removal, Stemming, Lemmatization, x["review"]), axis=1)
        vectorized_df = Vectorize(Vectorization, Ngram, df2["review"])
        print(f"Shape of dataframe(Stopword_Removal={Stopword_Removal}, Stemming={Stemming}, Lemmatization={Lemmatization}, Vectorization={Vectorization}, Ngram={Ngram}) is {vectorized_df.shape}")

Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=COUNT, Ngram=(1, 1)) is (20000, 19048)
Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=COUNT, Ngram=(2, 2)) is (20000, 169588)
Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=COUNT, Ngram=(3, 3)) is (20000, 339752)
Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=TFIDF, Ngram=(1, 1)) is (20000, 19048)
Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=TFIDF, Ngram=(2, 2)) is (20000, 169588)
Shape of dataframe(Stopword_Removal=False, Stemming=False, Lemmatization=False, Vectorization=TFIDF, Ngram=(3, 3)) is (20000, 339752)
Shape of dataframe(Stopword_Removal=False, Stemming=True, Lemmatization=False, Vectorization=COUNT, Ngram=(1, 1)) is (20000, 13675)
Shape of dataframe(Stopword_Removal=False, Stemming=True, Lemmatiz